In [1]:
import pandas as pd

In [2]:
FSIS_DATA = "fsis-processors-with-location.csv"

In [3]:
df = pd.read_csv(FSIS_DATA)

In [4]:
df.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
0,0,M267,4802,JBS,JBS Tolleson Inc.,AZ,Large,Beef,5.0,4.0,"651 S. 91st Ave, Tolleson, AZ 85353",33.441660,-112.252559
1,1,M354,5117,Cargill,Cargill Meat Solutions,CA,Large,Beef,5.0,4.0,"3115 S. Fig Ave., Fresno, CA 93706",36.689651,-119.801950
2,2,M6063A,5289,Central Valley Meat Company,"Central Valley Meat Co., Inc.",CA,Large,Beef,5.0,4.0,"10431 8 3/4 Ave., Hanford, CA 93230",36.321273,-119.612222
3,3,M783 + P783 + V783,5144,Central Valley Meat Company,Harris Ranch Beef Company,CA,Large,Beef,5.0,4.0,"16277 S McCall Ave., Selma, CA 93662",36.499212,-119.614553
4,4,M21488 + V21488,8349,Independent,OWB Packers LLC,CA,Large,Beef,4.0,4.0,"57 East Shank Road, Brawley, CA 92227",33.000969,-115.521786


In [7]:
df_chickens = df[df['Animals Processed'].str.contains('Chicken')]

In [9]:
df_chickens.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
71,71,P1317 + V1317,4495,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"700 McDonald Avenue, Albertville, AL 35950",34.260726,-86.203222
72,72,P7485 + V7485,4518,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"1020 County Road 114, Jack, AL 36346",31.500628,-85.903438
73,73,P912,4493,Cargill,"Wayne Farms, LLC",AL,Large,Chicken,5.0,5.0,"444 Baskin Street South, Union Springs, AL 36089",32.139166,-85.721704
74,74,P7342 + V7342,4516,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"808 Ross Clark Circle NE, Dothan, AL 36303",31.225754,-85.362068
75,75,P1235,4509,Cargill,Wayne Farms LLC,AL,Large,Chicken,5.0,5.0,"254 Ipsco Road, Decatur, AL 35601",34.607601,-87.042595


In [10]:
import folium

In [23]:
m = folium.Map(location=[df_chickens.iloc[0]['latitude'], df_chickens.iloc[0]['longitude']])

In [24]:
circles = []
for index, row in df_chickens.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(m)
    circle = folium.Circle(location=[row['latitude'], row['longitude']], radius=50*1609.34, fill=False, color='red')
    circle.add_to(m)
    circles.append(circle)

In [25]:
m

In [29]:
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from pyproj import Geod
from shapely.geometry import Point, LineString

line_string = LineString([Point(-1.154321, 55.124412), Point(-1.2354352, 56.2345235)])
geod = Geod(ellps="WGS84")
geod.geometry_length(line_string)


123700.61484174021

In [32]:
polygons = []
geod = Geod(ellps="WGS84")
# check for circle intersections
for i in range(len(circles)):
    for j in range(i+1, len(circles)):
        c1 = circles[i]
        c2 = circles[j]
        point1 = Point(c1.location[1], c1.location[0])
        point2 = Point(c2.location[1], c2.location[0])

        line_string = LineString([point1, point2])
        dist = geod.geometry_length(line_string)/1609.34

        print(dist)
        if dist < 100: # there is some overlap if distance is less than the radius of the circles
            intersection = Polygon(c1.intersections(c2).get_bounds())
            polygons.append(intersection)
        break

190.99994442141886
45.275702925434125


AttributeError: 'Circle' object has no attribute 'intersections'